# Демонстрация BERT4Rec в контексте музыкальных рекомендаций

Архитектура представленной модели основана на архитектуре BERT4Rec, назначение модели — продолжение последовательностей.
Модель обладает кастонмным эмбеддинг модулем, учитывающим автора, альбом и содержание каждой композиции в последовательности, выраженное в наборе числовых характеристик (темп, громкость, тональность и т.п.).
Данный блокнот демонстрирует процесс обучения и валидации модели на демонстрационном датасете и выполняет инференс модели на демострационной последовательности.

## Обучение модели

In [ ]:
from train import train_model
from config import config, demo_sample_meta, CURRENT_DIR

Функция train_model производит все необходимые действия для обучения модели с возможностью ее дальнейшей валидации. Использует словарь config с гиперпараметрами модели, конфигурацией процесса обучения.
Функция создаст необходимые папки внутри рабочей дериктории и будет сохранять в нее логи и промежуточные состояния модели и оптимизатора.
В рамках демонстрации будет использована конфигурация модели из словаря demo_sample_meta модуля config.
В процесс обучения обработается 5 эпох. 

In [ ]:
demo_config = config.copy()

# Заполнение демонстрационного конфига
demo_config['playlists_amo']        = demo_sample_meta['playlists_amo']        
demo_config['unique_songs']         = demo_sample_meta['unique_songs']         
demo_config['unique_artists']       = demo_sample_meta['unique_artists']       
demo_config['unique_albums']        = demo_sample_meta['unique_albums']        
demo_config['numeric_features_amo'] = demo_sample_meta['numeric_features_amo'] 
demo_config['num_epochs']           = demo_sample_meta['num_epochs']

demo_config['dataset_path']         = demo_sample_meta['demo_dataset_path']
demo_config['demo_sequence_path']   = demo_sample_meta['demo_sequence_path']
demo_config['demo_names_path']      = demo_sample_meta['demo_names_path']


In [ ]:
print('===< BERT4MusicRec >===')
train_model(demo_config)

## Валидация модели

На этом этапе происходит визуализация прогресса обучения с целью определения наилучшего состояния модели.

In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter1d

In [ ]:
# Загружаем логи
ea = event_accumulator.EventAccumulator(config['writer_dict'])
ea.Reload()

# Посмотреть, какие теги логировались
print(ea.Tags())

### Вывод графика ошибки

In [ ]:
# Получаем значения по тегу
events = ea.Scalars('train_loss')
steps = [e.step for e in events]
values = [e.value for e in events]

# Сглаживание значений с помощью гауссова фильтра
smoothed_values = gaussian_filter1d(values, sigma=10)

# Построение графика
plt.figure(figsize=(10, 4))

plt.plot(steps, values, label='Loss (raw)', linewidth=1, alpha=0.4, color='blue')

plt.plot(steps, smoothed_values, label='Loss (smoothed)', linewidth=2, color='red')

plt.xlabel('Батч')
plt.ylabel('Ошибка')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Вывод динамики валидационных метрик

In [ ]:
metrics = ea.Tags()['scalars']
metrics.remove('train_loss')

data = {}

for metric in metrics:
    # Получаем все события для данной метрики
    events = ea.Scalars(metric)
    steps = [e.step for e in events]
    values = [e.value / 16 for e in events]
    data[metric] = (steps, values)

# Построение графика
plt.figure(figsize=(9, 4))

for metric, (steps, values) in data.items():
    plt.plot(steps, values, label=metric, linewidth=1.5)

all_steps = np.arange(min(steps), max(steps)+1)
plt.gca().set_xticks(all_steps)

plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=1)

plt.xlabel("Эпоха")
plt.ylabel("Значение метрики")
plt.legend()
plt.grid(True)
plt.show()

## Генерация рекомендаций

На этом этапе будет произведен инференс модели на демонстрационной последовательности. Так как определение лучшего состояния демонстрационной модели задача бессмысленная, загрузим в память модель полученную в конце пятой эпохи.

In [ ]:
from flask_app import load_model, generate_recommendations, form_inference_sequence
import torch
import pandas as pd

In [ ]:
# Загрузка модели в память
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_model(demo_config, device, 5)

# Загрузка демонстрацинной последовательности
seq = pd.read_csv(demo_config['demo_sequence_path'])

In [ ]:
# Генерация рекомендаций
recommendations = generate_recommendations(model, form_inference_sequence(seq, device))
recommendations

### Визуализация рекомендаций

In [ ]:
# Объединение демонтрационной последовательности и массива рекомендаций
final_playlist = pd.concat([
    seq[['song_id']],
    pd.DataFrame([0], columns=['song_id']),
    pd.DataFrame(recommendations, columns=['song_id'])
])

In [ ]:
# Названия композиций, исполнителей и альбомов
names   = pd.read_csv(demo_config['demo_names_path'])

# Добавление названий 
pd.merge(final_playlist, names, how='left', on='song_id')